In [1]:
import sys, os
from pathlib import Path
sys.path.append(os.path.abspath('./..'))
import utils

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# gdrive = Path('/Users/psr/Library/CloudStorage/GoogleDrive-paru@stanford.edu')
# datadir = gdrive / 'My Drive/NMBL Lab/opencap_nmd/data/2023-05_dhd'
datadir = Path('../datadir')

df_session_now = pd.read_excel(datadir / 'session_info.xlsx')
df_trial_now = pd.read_excel(datadir / 'trial_info.xlsx')
df_part_now = pd.read_excel(datadir / 'participant_info.xlsx')
# df_session_now = pd.read_excel(datadir / 'temp/session_info.xlsx')
# df_trial_now = pd.read_excel(datadir / 'temp/trial_info.xlsx')
# df_part_now = pd.read_excel(datadir / 'temp/participant_info.xlsx')

df_part_now.date = pd.to_datetime(df_part_now.date)
df_part_now.date = df_part_now.date.dt.strftime('%Y-%m-%d')


In [2]:
sessions = utils.getUserSessions()
sessions_all = []
for s in tqdm(sessions):
    sessions_all.append(utils.getSessionJson(s['id']))
sessions = sessions_all


  0%|          | 0/385 [00:00<?, ?it/s]

In [3]:
part_labels = ['type', 'clinician', 'weight', 'height', 'age', 'sex',
               'gender', 'data_sharing', 'orthotics', 'clock_a', 'brooke',
               'time_10mwt', 'time_10mwrt', 'time_tug_line', 'time_tug_cone',
               'clock_b', 'time_5xsts', 'clock_c', 'time_stairs_up', 'notes_p']


In [4]:
meta = [s for s in sessions]
session_data = []
part_data = []
trial_data = []
for s in tqdm(meta, leave=False):
    sid = s['id']
    pid = s['name']
    session_data.append(dict(
        sid=sid,
        pid=pid,
        created_at=s['created_at'],
        server=s['server'],
        fps=s['meta']['settings']['framerate'] if 'settings' in s['meta'].keys() else np.nan,
        notes_s=np.nan,
    ))
    for t in s['trials']:
        trial = t['name']
        if trial in ('neutral', 'calibration'):
            continue
        trial_data.append(dict(
            sid=sid,
            pid=pid,
            created_at=t['created_at'],
            trial=trial,
            trial_clean=np.nan,
            notes_t=np.nan,
        ))
    # tz = 'America/New_York' if pid.startswith('mdf_') else 'America/Los_Angeles'
    tz = 'America/Los_Angeles'
    part = dict(
        pid=pid,
        date=pd.to_datetime(s['created_at']).tz_convert(tz).strftime('%Y-%m-%d')
    )
    part.update({pl:'' for pl in part_labels})
    part_data.append(part)

df_session = pd.DataFrame(session_data).sort_values('pid')
df_part = pd.DataFrame(part_data).sort_values('pid').drop_duplicates()
df_trial = pd.DataFrame(trial_data).sort_values(['pid', 'trial'])

df_part.shape, df_session.shape, df_trial.shape


  0%|          | 0/385 [00:00<?, ?it/s]

((177, 22), (385, 6), (1708, 6))

In [5]:
# def get_pid(s):
#     name = s['name'].lower()
#     if len(name) == 4 and name[1:].isnumeric() and name[0]=='p':
#         if int(name[1:]) < 10:
#             return ''
#         return name
#     if name[:4] == 'mdf_' and name[4:7].isnumeric():
#         return name
#     return name

# meta = [s for s in sessions if get_pid(s)]
# session_data = []
# part_data = []
# trial_data = []
# for s in tqdm(meta, leave=False):
#     sid = s['id']
#     pid = get_pid(s)
#     # if int(pid[1:]) < 10:
#     #     continue
#     session_data.append(dict(
#         sid=sid,
#         pid=pid,
#         created_at=s['created_at'],
#         server=s['server'],
#         fps=s['meta']['settings']['framerate'],
#         notes_s=np.nan,
#     ))
#     for t in s['trials']:
#         trial = t['name']
#         if trial in ('neutral', 'calibration'):
#             continue
#         trial_data.append(dict(
#             sid=sid,
#             pid=pid,
#             created_at=t['created_at'],
#             trial=trial,
#             trial_clean=np.nan,
#             notes_t=np.nan,
#         ))
#     tz = 'America/Los_Angeles' if pid[0] == 'p' else 'America/New_York'
#     part = dict(
#         pid=pid,
#         date=pd.to_datetime(s['created_at']).tz_convert(tz).strftime('%Y-%m-%d')
#     )
#     part.update({pl:'' for pl in part_labels})
#     part_data.append(part)
        
# df_session = pd.DataFrame(session_data).sort_values('pid')
# df_part = pd.DataFrame(part_data).sort_values('pid').drop_duplicates()
# df_trial = pd.DataFrame(trial_data).sort_values(['pid', 'trial'])

# df_part.shape, df_session.shape, df_trial.shape


In [6]:
df_part.pid.unique()


array(['CMT001', 'CMT004', 'CMT005', 'CMT008', 'CMT009', 'CMT011',
       'CMT012', 'CMT013', 'CMT016', 'CMT017', 'CMT018', 'CMT021',
       'CMT022', 'CMT023', 'CMT024', 'CMT028', 'CMT034', 'CMT036',
       'CMT_test', 'DB-161', 'DB-294', 'DB-324', 'DB-334', 'DB-339',
       'DB-341', 'DB-345', 'DB-346', 'DB-350', 'DB-356', 'DB-371',
       'DB-397', 'DB-404', 'DB-484', 'DB-492', 'DB-515', 'DB-538',
       'DB-614', 'DB-615', 'DB-618', 'DB-619', 'DB-620', 'DB-621',
       'DB-622', 'DB-623', 'DB-624', 'DB-625', 'DB-626', 'DB-627',
       'DB-628', 'DB-629', 'DB-630', 'DB-631', 'DB-632', 'DB-633',
       'DB-634', 'DB-635', 'MDA10', 'MDA3', 'MDA5', 'MDA6', 'MDA7',
       'MDA8', 'cmt002', 'cmt006', 'cmt010', 'cmt014', 'cmt015', 'cmt025',
       'cmt026', 'cmt027', 'cmt043', 'constance_test', 'new_test', 'p001',
       'p201', 'p202', 'p203', 'p212', 'p213', 'p214', 'p215', 'p216',
       'p217', 'p218', 'p219', 'p221', 'p222', 'p223', 'p224', 'p225',
       'p226', 'p227', 'p228', 'p22

In [7]:
df_part_new = df_part_now.set_index(['pid', 'date']).combine_first(df_part.set_index(['pid', 'date']))
cols = list(df_part_now.columns)
cols += [c for c in df_part.columns if c not in cols]
df_part_new = df_part_new.reset_index()[cols]
df_part_new.sort_values(['date', 'pid'], inplace=True)
df_part_new.reset_index( drop=True, inplace=True)
df_part_new


,pid,date,type,location,clinician,weight,height,age,birth_year,sex,...,time_10mwrt,time_tug_line,time_tug_cone,clock_b,time_5xsts,clock_c,time_stairs_up,unable,notes_p,todo
0,58385b9a,2022-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No REDCap; uses arms on STS,NaN
1,a31757fd,2022-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No REDCap,NaN
2,3d17b3ee,2022-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test subject,NaN
3,6ec44398,2022-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No OpenCap trials,NaN
4,7e6bd5de,2022-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No OpenCap trials,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,DB-161,2025-03-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
576,281_test,2025-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
577,DB-184,2025-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
578,DB-185,2025-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_session_new = df_session_now.set_index(['pid', 'created_at']).combine_first(df_session.set_index(['pid', 'created_at']))
cols = list(df_session_now.columns)
cols += [c for c in df_session.columns if c not in cols]
df_session_new = df_session_new.reset_index()[cols]
df_session_new.sort_values(['created_at', 'pid'], inplace=True)
df_session_new.reset_index( drop=True, inplace=True)
df_session_new


# df_session_new = df_session_now.set_index('sid').combine_first(df_session.set_index('sid'))
# cols = ['pid', 'created_at']
# cols += [x for x in df_session_new.columns if x not in cols]
# cols = [x for x in cols if x != 'notes_s'] + ['notes_s']
# df_session_new = df_session_new[cols]
# df_session_new.sort_values('pid', inplace=True)
# df_session_new.reset_index(inplace=True)
# df_session_new


,sid,created_at,pid,url,fps,server,notes_s
0,017ca154-7573-4cd1-ae3f-42dfb8c4f9b0,2023-05-13T21:16:17.904194Z,p012,https://app.opencap.ai/session/017ca154-7573-4...,60.0,171.65.92.206,NaN
1,ee23fbb3-a991-4aa4-9a2f-a213ec9ec6c5,2023-05-13T21:24:25.274931Z,p011,https://app.opencap.ai/session/ee23fbb3-a991-4...,120.0,54.190.39.224,NaN
2,64ac99c8-f9d2-425d-b990-6d770c0288cd,2023-05-13T21:56:04.599875Z,p012,https://app.opencap.ai/session/64ac99c8-f9d2-4...,60.0,171.65.92.206,NaN
3,d6b90c12-92a9-4e5b-9500-54655dde7e63,2023-05-13T21:57:56.146248Z,p012,https://app.opencap.ai/session/d6b90c12-92a9-4...,120.0,171.66.229.49,NaN
4,057d10da-34c7-4fb7-a127-6040010dde06,2023-05-13T22:24:42.055213Z,p013,https://app.opencap.ai/session/057d10da-34c7-4...,60.0,54.190.39.224,NaN
...,...,...,...,...,...,...,...
803,add6c555-02e2-4b33-b303-2c5701327099,2024-11-02T22:18:30.228921Z,DB-339,NaN,60,171.65.102.146,NaN
804,0f7ad50b-25db-45b5-97cf-c74e5e1b6434,2024-11-02T22:24:48.158847Z,MDA10,NaN,120,18.236.88.13,NaN
805,190ee0a1-b2d2-4bf4-8dfb-83869f60081d,2024-11-02T22:34:45.886631Z,MDA10,NaN,60,18.236.88.13,NaN
806,ec90913c-8e22-468c-a9cc-8b10dd1918dd,2025-02-11T22:11:09.910576Z,DB-397,NaN,120,18.236.88.13,NaN


In [9]:
df_trial_new = df_trial_now.set_index(['sid', 'trial']).combine_first(df_trial.set_index(['sid', 'trial']))
cols = list(df_trial_now.columns)
cols += [c for c in df_trial.columns if c not in cols]
df_trial_new = df_trial_new.reset_index()[cols]
df_trial_new.sort_values(['pid', 'created_at'], inplace=True)
df_trial_new.reset_index( drop=True, inplace=True)
df_trial_new

# df_trial_new = df_trial_now.set_index(['sid', 'trial']).combine_first(df_trial.set_index(['sid', 'trial']))
# df_trial_new.reset_index(inplace=True)
# df_trial_new.sort_values(['pid', 'created_at'], inplace=True)
# cols = ['pid', 'sid', 'created_at', 'trial', 'trial_clean']
# cols += [x for x in df_trial_new.columns if x not in cols]
# cols = [x for x in cols if x != 'notes_t'] + ['notes_t']
# df_trial_new = df_trial_new[cols]
# df_trial_new


,pid,created_at,sid,trial,trial_clean,score,notes_t,bug_status,bug_description,parker/sydney_todo,parker_action
0,281_test,2025-03-07T20:32:44.951522Z,ddfc2b9e-e6b5-4b83-8e35-32cc4d0e7252,brooke,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,281_test,2025-03-07T20:34:30.043831Z,ddfc2b9e-e6b5-4b83-8e35-32cc4d0e7252,fake_pass,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,281_test,2025-04-02T18:07:59.035182Z,924784b5-5ef7-4e62-afa9-14501e0fed62,gait,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3d17b3ee,2022-03-02T21:52:49.550000Z,3d17b3ee-3595-4ea4-ba62-d277fa59a273,STS,NaN,NaN,test recording,NaN,NaN,NaN,NaN
4,3d17b3ee,2022-03-02T21:53:31.464000Z,3d17b3ee-3595-4ea4-ba62-d277fa59a273,TUG,NaN,NaN,test recording,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
5217,testnoirin,2022-08-24T17:20:04.331078Z,65313f98-2457-459e-9455-729be1fb6630,10MWRTtest1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5218,testnoirin,2022-08-24T17:20:43.856434Z,65313f98-2457-459e-9455-729be1fb6630,Squats,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5219,training_test,2023-11-22T22:04:03.596526Z,0ace947e-9e1d-409f-9e14-67690e73f560,10MWT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5220,training_test,2023-11-22T22:04:57.107987Z,0ace947e-9e1d-409f-9e14-67690e73f560,10MRT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_part_new.to_excel(datadir / 'temp/participant_info.xlsx', index=False)
df_session_new.to_excel(datadir / 'temp/session_info.xlsx', index=False)
df_trial_new.to_excel(datadir / 'temp/trial_info.xlsx', index=False)
